# Extração dos dados

#### **Deletar** antes de subir no Github

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


---

## 1 Importando bibliotecas

Carregando bibliotecas necessárias

In [ ]:
import requests
import pandas as pd
from io import BytesIO

Criando função para gerar os dados da APIs

In [ ]:
# onde codigos é uma lista, url_base é uma string e tipo_url pode ser simples (sem filtros) ou composta (com filtros)
def gerarDataFrame(url_base, tipo_url, codigos, nova_coluna):
    
    df = pd.DataFrame()
    session = requests.Session()
    
    if tipo_url == "simples":
        r = session.get(url = url_base)
        df = df.append(pd.json_normalize(r.json(), record_path =['value']), ignore_index = True)
        df.reset_index(drop=True)
    else:
        for codigo in codigos:
            url_final = url_base + codigo + "'&$top=10000&$format=json"
            r = session.get(url = url_final)
            df_temp = pd.json_normalize(r.json(), record_path =['value'])
            df_temp[nova_coluna] = codigo
            df = df.append(df_temp)
    
    df.reset_index(drop=True)
    return df

Função para gerar os arquivos CSV, semetre por semestre [desse link](https://dados.gov.br/dataset/ranking-de-instituicoes-por-indice-de-reclamacoes/resource/1831c634-9055-4687-8faa-be9c9b811912).

In [ ]:
# gerar arquivos CSV
def gerarCSV(lista_anos, lista_trimestre):
  df = pd.DataFrame()
  session = requests.Session()

  for ano in lista_anos:
      for trimestre in lista_trimestre:
        url_base = 'https://www3.bcb.gov.br/rdrweb/rest/ext/ranking/arquivo?ano=' + str(ano) + '&periodicidade=TRIMESTRAL&periodo=' + str(trimestre) + '&tipo=Bancos+e+financeiras'
        
        r = session.get(url_base).content
        df = df.append(pd.read_csv(BytesIO(r), encoding='latin1', delimiter=';'))

  return df

## 2 Gerandos os dados

URL dos dados da API e caminho salvamento

In [ ]:
url_grupos_consolidados = "https://olinda.bcb.gov.br/olinda/servico/Informes_ListaTarifasPorInstituicaoFinanceira/versao/v1/odata/GruposConsolidados?$top=10000&$format=json"
url_lista_instituicoes = "https://olinda.bcb.gov.br/olinda/servico/Informes_ListaTarifasPorInstituicaoFinanceira/versao/v1/odata/ListaInstituicoesDeGrupoConsolidado(CodigoGrupoConsolidado=@CodigoGrupoConsolidado)?@CodigoGrupoConsolidado='"
url_tarifas = "https://olinda.bcb.gov.br/olinda/servico/Informes_ListaTarifasPorInstituicaoFinanceira/versao/v1/odata/ListaTarifasPorInstituicaoFinanceira(PessoaFisicaOuJuridica=@PessoaFisicaOuJuridica,CNPJ=@CNPJ)?@PessoaFisicaOuJuridica='F'&@CNPJ='"

# caminho para salvamento dos arquivos
caminho = '/content/drive/MyDrive/Colab Notebooks/dados/'

Obtendo o dados dos Grupos Consolidados

In [ ]:
df_consolidado = gerarDataFrame(url_grupos_consolidados, 'simples', [], '')

Obtendo a lista de todas as Instituições com base no Grupo Consolidado (df_consolidado)

In [ ]:
# lista dos códigos dos grupos
lista_codigo_grupo = df_consolidado.Codigo.tolist()

In [ ]:
# obtendo lista de todas as instituições
df_lista_inst = gerarDataFrame(url_lista_instituicoes, 'composta', lista_codigo_grupo, 'CodigoGrupo')

Obtendo lista de Tarifas para Pessoas Fisicas

In [ ]:
# lista dos CNPJs das instituições
lista_cnpjs = df_lista_inst.Cnpj.tolist()

In [ ]:
# obtendo lista de todas as tarifas (demora para processar tudo)
df_tarifas = gerarDataFrame(url_tarifas, 'composta', lista_cnpjs, 'CNPJ')

Obtendo a lista de reclamações

In [ ]:
# Criando a lista de anos e de trimestres
lista_anos = [2020, 2021]
lista_trimestre = [1, 2, 3, 4]

In [ ]:
#obtendo lista de todas as reclamações dos anos e semestres acima
df_reclamações = gerarCSV(lista_anos, lista_trimestre)

## 3 Salvando os dados em um diretório

In [ ]:
df_tarifas.to_csv(caminho + '/df_tarifas.csv', encoding='latin1')
df_lista_inst.to_csv(caminho + '/df_lista_inst.csv', encoding='latin1')
df_consolidado.to_csv(caminho + '/df_consolidado.csv', encoding='latin1')
df_reclamações.to_csv(caminho + '/df_reclamações.csv', encoding='latin1')